# Introduction to the JWST pipeline

#### This is a tutorial on running the JWST Science Calibration Pipeline (referred to as “the pipeline”).

## Pre-notebook setup: <a class="anchor" id="0"></a>

Before going through the examples in this notebook, complete the following setup steps.

1. This tutorial assumes a user has [conda](https://conda.io/projects/conda/en/latest/user-guide/install/index.html) installed for package and environment management. For the purpose of this tutorial we will install JWST in a new conda environment named `jwst-test`, using Python 3.8. The JWST package installs its dependencies but we need to install Python and numpy in this conda environment.

- #### Make and activate the new conda environment

    *% conda create -n jwst-test python=3.8*

    *% source activate jwst-test*

- #### Install numpy

    *% pip install numpy==1.19*

- #### Install jupyter (for ipython and jupyter notebooks).

    *% pip install jupyter*

    After you create the conda environment and install the packages, exit and reopen this notebook, ensuring that you have the new environment activated.


2. The examples in this notebook use sample data and reference/parameter files. Before attempting any examples in this notebook, download the contents of LINK_TO_BOX to the directory containing this notebook.


3. CRDS, the management system for the reference and parameter files that the pipeline requires, needs to have certain environment variables set to know which server to connect to to download these files, and a local directory. In your shell:

    *% export CRDS_PATH=$HOME/crds_cache*

    *% export CRDS_SERVER_URL=https://jwst-crds.stsci.edu*

## TOC:

* [Overview of JWST Pipeline](#1)
    * [What is the JWST Pipeline?](#1a)
    * [Reference Files and CRDS](#1b)
    * [Parameter Files](#1c)
* [Installation](#2)
    * [Installing the Last Released Verion of JWST](#2a)
    * [Installing the Latest Development Version from Github](#2b)
* [Running the Pipeline in Python](#3)
    * [Overview of Running the Pipeline in Python](#3a)
    * [Example 1: Running a Full Pipeline in Python](#3b)
    * [Example 2: Running a Single Pipeline Step in Python](#3c)
    * [Options when Running the Pipeline in Python: 'call' vs 'run' Methods](#3d)
        * [Example 3: Using `run` Instead of `call` ](#3da)
    * [Configuring the Pipeline in Python](#3e)
        * [Setting Parameters](#3f)
            * [Option 1: Using a Custom or Modified Parameter File (Reccomended)](#3f1)
            * [Option 2: Setting Parameters as Attributes on a Pipeline/Step Directly](#3f2)
            * [Option 3: Setting Parameters via Dictionary when Using the `call` Method](#3f3)
            * [More on Configuring Parameters in Python](#3f4)
        * [Changing Reference Files](#3g)
            * [Option 1: Overriding Reference Files as an Argument to A Pipeline/Step](#3g1)
            * [Option 2: Overriding Reference Files on a Pipeline/Step Directly](#3g2)
        * [Skipping Steps](#3h)
            * [Option 1: Specifying Pipeline Steps to Skip within a Parameter File](#3h1)
            * [Option 2: Skipping Steps Directly when using 'run'](#3h2)
* [Running the Pipeline with the Command Line Interface (CLI)](#4)
    * [Example 3: Running a Full Pipeline using the CLI](#4a)
    * [Example 4: Running a Single Pipeline Step using the CLI](#4b)
    * [Configuring the Pipeline using the Command Line Interface](#4c)
        * [Setting Parameters](#4d)
        * [Changing Reference Files](#4e)
        * [Skipping Steps](#4f)
    * [Additional Command Line Functionality](#4g)
* [FAQ](#5)

# Overview of JWST Pipeline <a class="anchor" id="1"></a>

The following will provide a very brief summary to provide context for some of the information in this notebook: a more extensive description is available in the [JWST pipeline documentation](https://jwst-pipeline.readthedocs.io/en/latest/).

## What is the JWST Pipeline? <a class="anchor" id="1a"></a>

The JWST pipeline is the software suite that processes raw data from the JWST instruments to make it science-ready by performing various corrections and operations on the data. The pipeline is written in Python and hosted on [Github]((https://github.com/spacetelescope/jwst).

Data products available for download in the MAST archive have already been processed with this pipeline, but users wishing to do custom processing may want to configure and run them pipeline themselves. The pipeline can be ran and configured in [Python](#3) or via the [command line interface](#4) `strun`.

The JWST pipeline is broken into 3 [stages](https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/main.html). Each stage consists of a series of steps. 

- Stage 1: Detector-level corrections and ramp fitting for individual exposures

  The first stage, called `calwebb_detector1`, is applied nearly universally for all instruments and modes. It consists of detector-level corrections that are performed on a group-by-group basis, followed by ramp fitting. The output of stage 1 processing is a countrate image per exposure, or per integration for some modes. The input to this step is `uncal.fits` file, and the output is a `rate.fits` file.


- Stage 2: Instrument-mode calibrations for individual exposures

  The second stage is split into separate modules for imaging and spectroscopic data, called `calwebb_image2` and `calwebb_spec2` respectively. It consists of additional instrument-level and observing-mode corrections and calibrations to produce fully calibrated exposures. The details differ for imaging and spectroscopic exposures, and there are some corrections that are unique to certain instruments or modes. The input to this step is `rate.fits` file, and the output is a `cal.fits` file.
  
  
- Stage 3: Combining data from multiple exposures within an observation

  The third stage is divided into five separate modules depending on the observing mode. It consists of routines that work with multiple exposures and in most cases produce some kind of combined product. There are unique pipeline modules for stage 3 processing of imaging, spectroscopic, coronagraphic, AMI, and TSO observations.
  
Each dataset, depending on what type of observation it is, will have three stages in its pipeline. Furthermore, each pipeline consists of individual Steps (e.g `AssignWCSStep`, `LinearityStep`). The modular nature of the pipeline allows users to customize their processing by running the full pipeline, a single stage, a single step within each stage, etc. Steps can also be 'switched' on and off during processing (for example, running the `Detector1Pipeline` but omitting the `DarkCurrentStep`).
  
All pipelines and steps have parameters that can be set to change various aspects of how they execute.The main way in which the pipeline is configured is via `Parameter Files`. `Reference Files` provide the data for the multitude of calibrations and corrections performed by the pipeline. Both reference files and parameter files can be customized when running the pipeline.

## Reference Files and CRDS <a class="anchor" id="1b"></a>

Reference files are data files - seperate from the input data being processed - that the JWST pipeline requires to calibrate data and apply necessary corrections and calibrations to achieve science-ready data. An example of a reference file is a dark-current correction file, which is an array that represents the estimated dark-current for each pixel in an image. Each data set has a specific set of up-to-date reference files associated with it which supply the data for all the pipeline calibration steps.

Reference files are created and validated by the JWST instrument teams. Because many of these corrections are time dependent (e.g a monthly dark file), or are periodically updated and improved as understanding of the instrument improves, they must be version controlled to ensure users can access the exact set of files for a dataset as well as revert back to previous versions if needed. Managing these files and determining the exact set of reference files to apply to any given data file is not a trivial task: the CRDS (Calibration Reference Data System) manages these intricacies and is the interface for obtaining and managing pipeline reference files.

Users internal to the STScI network already have all reference files on disk in the CRDS file system on central storage and will not have to do any special setup to allow the pipeline access to reference files. External users will have to configure their environment to point to the CRDS_SERVER_URL (to download reference files from this server) as well as a directory in their local filesystem (CRDS_PATH) to which these files will be downloaded to, and which the pipeline will recognize as the location of these files when run (this should have been done in the [pre-notebook setup](#0)) In your shell, set the environment variables:

##### *% export CRDS_PATH=$HOME/crds_cache*

##### *% export CRDS_SERVER_URL=https://jwst-crds.stsci.edu*

When you run the pipeline on a dataset for the first time, the CRDS code within the pipeline will determine the best set of reference files based on the exposure date, observation parameters (e.g instrument type and mode, filter), and 'context' (akin to a snapshot in time, the most up-to-date being the default, but users can also roll back to a previous context with different mappings). Then, it will connect to the server (specified via `CRDS_SERVER_URL`) and download those files to your local directory `CRDS_PATH`. Subsequent runs of the Python will first check `CRDS_PATH` for existing required reference files so they aren't re-downloaded. 

### Parameter Files <a class="anchor" id="1b"></a>
Parameter files are [ASDF](https://readthedocs.org/projects/asdf/) format files that set the multitude of parameters for pipeline steps, and are the main way that the JWST pipeline is configured. They are similar to reference files in the sense that they are created by JWST instrument teams and direct the pipeline behavior/output. Also like reference files, they are version controlled and managed by CRDS. This version-controlling gives the instrument teams the ability to evolve parameter values for different observing modes over time.

An example of a parameter file is:

        >> jwst_miri_pars-detector1pipeline_0001.asdf

When a user runs the pipeline, the corresponding parameter files for the step(s) being run and the dataset being processed will be downloaded to the local crds cache (CRDS_PATH) along side the other reference files. 

Note that not every observing mode/pipeline step have a parameter file - in some cases, there are no special parameter selections needed and the coded defaults suffice. For example, the `Image2Pipeline` for Nircam does not have a parameter file, and the coded default values are used for every dataset. See the [JWST CRDS website](https://jwst-crds.stsci.edu/) to browse parameter files for different steps and observing modes.

Parameter files may be ignored or overridden when running the pipeline. Users can override parameters individually, or supply their own custom parameter file which will override keywords in any parameter file existing in CRDS for that dataset. See \<link> for more information on configuring the pipeline. 

# Installation <a class="anchor" id="2"></a>
## Installing the Last Released Verion of JWST<a class="anchor" id="2a"></a>
The latest public release can be installed with one command.

*% pip install jwst*
## Installing the Latest Development Version from Github<a class="anchor" id="2b"></a>
To work with the absolute most up-to-date version of the pipeline, the `main` branch of the Github repository can be installed. The source code for the JWST pipeline is located on Github in the spacetelescope organization.

https://github.com/spacetelescope/jwst

To install the latest code from github use the command:

*% pip install git+https://github.com/spacetelescope/jwst*

Make sure to keep this install up-to-date because the state of this branch changes on a daily basis.

# Running the Pipeline in Python <a class="anchor" id="3"></a>

## Overview of Running the Pipeline in Python <a class="anchor" id="3a"></a>

All pipelines and steps are avaiable to import and run in a Python session, for example:

`from jwst.pipeline import Image3Pipeline`

or 

`from jwst.dark_current import DarkCurrentStep`

As will be demonstrated in the examples below, once imported these pipelines and steps can be configured either via parameter files or on the Python object directly, and run on input data files specified by path (or loaded into memory as `DataModels`).

## Example 1: Running a Full Pipeline in Python <a class="anchor" id="3b"></a>
In this example, the `Detector1Pipeline` will be run with all default parameters and reference files (as determined by CRDS) on a NIRCAM image. The file for this example is an `uncal` file, which is the naming convention for file yet to undergo any pipeline corrections.

First, import the desired pipeline (in this case Detector1Pipeline).

In [1]:
from jwst.pipeline import Detector1Pipeline

And next, the basic call to run the pipeline, using the `call` method with the input file as an argument.

By default, the final correction is returned in-memory only (though outputs from intermediate steps may be saved as output files). This can be changed by setting the optional `save_results` argument to True in the call. In this example, we will choose to write out the file and also to assign the output, which is a `DataModel` (an object representing the entire fits file), to the variable `result` so that the result of the pipeline is available immediatley in memory.

You will notice that while running the pipeline, there is informative logging information printed to the screen and in this instance, you'll also see the file `jw42424001001_01101_00001_nrca5_trapsfilled.fits` output in the cwd - this is an intermediate output from the [persistance](https://jwst-pipeline.readthedocs.io/en/latest/jwst/data_products/nonscience_products.html#charge-trap-state-data-trapsfilled) step of the pipeline.

In [2]:
# result = Detector1Pipeline.call('jw42424001001_01101_00001_nrca5_uncal.fits', save_results=True)

The calibrated data model `result` contains information about how the pipeline was run - for example, you can see in `model.meta.cal_step` which steps were run (COMPLETE) and which were not (SKIPPED).

In [4]:
# result.meta.cal_step.instance

You can also inspect the names of the reference files that were used:

In [5]:
# result.meta.ref_file.instance

## Example 2: Running a Single Pipeline Step in Python <a class="anchor" id="3c"></a>

In the first example, the full Detector1Pipeline was run on the input `uncal` date. We can run the steps that comprise this pipeline individually, for example, the `LinearityStep`. Running a pipeline step is done in the exact same way: import the step and use the `call` method with your data as input. In this case, we will not set `save_results` to True so no intermediate output will be written, only a `DataModel` with `LinearityStep` will be returned in memory to the variable `lin_corr_im`.

In [6]:
from jwst.linearity import LinearityStep

step = LinearityStep()

#lin_corr_im = step.call('jw42424001001_01101_00001_nrca5_uncal.fits')

2021-05-10 09:50:31,539 - stpipe.LinearityStep - INFO - LinearityStep instance created.


## Options when Running the Pipeline in Python: 'call' vs 'run' Methods <a class="anchor" id="3d"></a>

There are two basic options when running the pipeline in Python, and the differences arise in how they are configured to run.

1. Using the 'call' method on a pipeline/step, and using parameter files to configure it (as done in previous examples). For example:    

    `Detector1Pipeline.call('input_file.fits')` 
    
    When the `.call` method is called on a pipeline instance, a new instance of that
    pipeline is created internally. The values in the parameter file are set as
    attributes on this new instance, the pipeline is run with these values, and
    then it is disposed of and the final result is returned. This is the reccomended
    way to run the pipeline since it is intended to be configured via parameter files.

2. Configuring the pipeline/step insance directly by using its 'run' method. In this approach, the instance of the pipeline or step you create will have all parameters be the coded defaults, and to change them you set the attributes on the object directly.

    `Detector1Pipeline.run('input_file.fits')`
    
    Or, equivalently
    
    `Detector1Pipeline('input_file.fits')`
    
    When 'pipe.run' or simply 'pipe()' are called, the instance you created is directly
    used. So, any attributes set on that pipeline will be the ones used to direct the processing.
    The additional setup done in `call` to set the parameter file as
    attributes on the pipeline is not done, you will have to set each pipeline parameter
    individually as an attribute on the pipeline instance you created before running it.

It is reccomended that users stick to using the 'call' method which uses parameter files to set pipeline options, but configuring it directly and using `run` is also an option. 

### Example 3: Using `run` Instead of `call`  <a class="anchor" id="3da"></a>
In this example, we will use the `run` method to run the `Image2Pipeline`. Parameters are set directly on the insance of `Image2Pipeline`, config files aren't used.

In [7]:
from jwst.pipeline import Image2Pipeline

# save_results has to be set here, because now we're calling this class directly 
pipe2 = Image2Pipeline(save_results=True)

#result2 = pipe2.run('jw42424001001_01101_00001_nrca5_rate.fits')

2021-05-10 09:50:35,628 - stpipe.Image2Pipeline - INFO - Image2Pipeline instance created.
2021-05-10 09:50:35,630 - stpipe.Image2Pipeline.bkg_subtract - INFO - BackgroundStep instance created.
2021-05-10 09:50:35,633 - stpipe.Image2Pipeline.assign_wcs - INFO - AssignWcsStep instance created.
2021-05-10 09:50:35,635 - stpipe.Image2Pipeline.flat_field - INFO - FlatFieldStep instance created.
2021-05-10 09:50:35,637 - stpipe.Image2Pipeline.photom - INFO - PhotomStep instance created.
2021-05-10 09:50:35,639 - stpipe.Image2Pipeline.resample - INFO - ResampleStep instance created.


Or, identically, calling the pipeline obj directly which invokes the `run` method.

In [ ]:
# result2 = pipe2('jw42424001001_01101_00001_nrca5_rate.fits')

## Configuring the Pipeline in Python <a class="anchor" id="3e"></a>


### Setting Parameters <a class="anchor" id="3f"></a>

As mentioned above, the parameter file is the basis of how pipelines/steps are configured - the parameter files in CRDS correspond to the 'best' set of parameters as chosen by the JWST instrument teams - but users are free to change any of these values when running the pipeline. There are a number of ways to do this when running the pipeline in Python.

Let's go through a few examples of this. The output `rate` file from the last example can be run through the [Image2Pipeline](https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_image2.html), which applies several calibtration steps including flat fielding and assigning a WCS. When this pipeline is run, the resulting data product is a `cal` file.

#### Option 1: Using a Custom or Modified Parameter File (Reccomended)<a class="anchor" id="3f1"></a>
The most straight forward way to configure the pipeline in Python is to change parameters within a parameter file, and pass this to the pipeline run with the `.call` method. Any CRDS parameter files will be overridden by those provided by the user.

Again, begin by importing the desired pipeline or step, in this case `Image3Pipeline`, and creating an instance of it `pipe3`:

In [9]:
from jwst.pipeline import Image3Pipeline
# pipe3 = Image3Pipeline()

If we run it just like this without making any modifications, the corresponding parameter file will be obtained from CRDS and those values will be used. If you look at the logging output, you'll see one of the first lines indicates that the file `jwst_nircam_pars-tweakregstep_0006.asdf` is being used.

In [10]:
# result3 = pipe3.call('jw42424001001_01101_00001_nrca5_cal.fits')

In the cwd, we have the file 'my_custom_parfile_tweakregstep.asdf'. This file is in the exact same format as jwst_nircam_pars-tweakregstep_0006.asdf, but some of the values for the tweakreg step have been changed. We can direct Image3Pipeline to run with these values instead by setting the config_file argument to the path of our custom refence file, in this case just config_file = my_custom_parfile_tweakregstep.asdf.

In [11]:
# result3_myparfile = pipe3.call('jw42424001001_01101_00001_nrca5_cal.fits',
                               # config_file='my_custom_parfile_tweakregstep.asdf')

#### Option 2: Setting Parameters as Attributes on a Pipeline/Step Directly<a class="anchor" id="3f2"></a>

Setting parameters as attributes directly only works when using the `run` method on a pipeline/step instance. Because using `run` directly does not recognize any parameter files (custom or chosen by CRDS), ALL parameters must be set as attributes or else the hard coded defaults (which aren't necessarily within a reasonable range for the particular observing mode of your data) will be used. In this example, we will set most of the `tweakreg` parameters to the same values that are in the corresponding parameter file, but change `brightest` and `sigma` to a a different value. 

The following is equivalent to the example above.


In [12]:
pipe3 = Image3Pipeline()


pipe3.brightest = 50
pipe3.kernel_fwhm = 2.302
pipe3.minobj = 15
pipe3.nclip = 2
pipe3.searchrad = 1.0
pipe3.separation = 0.5
pipe3.sigma = 3.0
pipe3.snr_threshold = 5


# pipe3.run('jw42424001001_01101_00001_nrca5_cal.fits')

2021-05-10 09:50:44,722 - stpipe.Image3Pipeline - INFO - Image3Pipeline instance created.
2021-05-10 09:50:44,724 - stpipe.Image3Pipeline.assign_mtwcs - INFO - AssignMTWcsStep instance created.
2021-05-10 09:50:44,726 - stpipe.Image3Pipeline.tweakreg - INFO - TweakRegStep instance created.
2021-05-10 09:50:44,729 - stpipe.Image3Pipeline.skymatch - INFO - SkyMatchStep instance created.
2021-05-10 09:50:44,731 - stpipe.Image3Pipeline.outlier_detection - INFO - OutlierDetectionStep instance created.
2021-05-10 09:50:44,734 - stpipe.Image3Pipeline.resample - INFO - ResampleStep instance created.
2021-05-10 09:50:44,736 - stpipe.Image3Pipeline.source_catalog - INFO - SourceCatalogStep instance created.


#### Option 3: Setting Parameters via Dictionary when Using the `call` Method<a class="anchor" id="3f3"></a>

It is possible to use the `call` method, which uses parameter files for configuration, and to change these values within a Python session without modifying or supplying any new parameter file. To do this, a nested dictionary specifiying any paramter overrides can be passed in. The following is also equivalent to the above examples.

According to the parameter precedence rule, parameters in the CRDS-specified parameter file will be used, so we only have to override the exact parameters we wish to change from that file (while in the previous example, every parameter had to be set to change them from the coded default values). 

In [13]:
pipe3 = Image3Pipeline()
#pipe3.call("jw42424001001_01101_00001_nrca5_cal.fits", 
#           steps={"tweakreg":{"brightest": 50, 'sigma': 3.0}})


2021-05-10 09:50:46,871 - stpipe.Image3Pipeline - INFO - Image3Pipeline instance created.
2021-05-10 09:50:46,873 - stpipe.Image3Pipeline.assign_mtwcs - INFO - AssignMTWcsStep instance created.
2021-05-10 09:50:46,876 - stpipe.Image3Pipeline.tweakreg - INFO - TweakRegStep instance created.
2021-05-10 09:50:46,878 - stpipe.Image3Pipeline.skymatch - INFO - SkyMatchStep instance created.
2021-05-10 09:50:46,881 - stpipe.Image3Pipeline.outlier_detection - INFO - OutlierDetectionStep instance created.
2021-05-10 09:50:46,883 - stpipe.Image3Pipeline.resample - INFO - ResampleStep instance created.
2021-05-10 09:50:46,885 - stpipe.Image3Pipeline.source_catalog - INFO - SourceCatalogStep instance created.


When running a single step rather than a pipeline, the parameters can be set individually.

In [14]:
from jwst.tweakreg import TweakRegStep

tweakreg = TweakRegStep()

#tweakreg.call("jw42424001001_01101_00001_nrca5_cal.fits", sigma=3.0, brightest=50)

2021-05-10 09:50:48,532 - stpipe.TweakRegStep - INFO - TweakRegStep instance created.


#### More on Configuring Parameters in Python<a class="anchor" id="3f4"></a>

The previous examples demonstrated the three possible ways of specifying pipeline parameters when running the pipeline in Pythom. It is reccomended that users use the first option, and specify any parameter overrides within a parameter file. 

Because there are multiple locations in which a parameter value may be specified (parameter file, coded defaults, setting the attributes on the pipe/step class directly, and command line paramater overriding which will be introduced later), the pipeline uses hierarchy rules to determine which value are used if they are specified in multiple locations.

1. Value specified in command line, or set on class directly when using ``pipe.run()`` or `pipe()`.
2. Value in user-provided parameter file.
3. Value in CRDS parameter file.
4. `Step`-coded default, determined by the parameter definition `Step.spec`

According to these rules, if you specify both a custom parameter file AND provide a dictionary of parameter values when running the pipeline using `call`, the dictionary values will override any in the parameter file. Additionally, if you pass in an incomplete parameter file, the CRDS file will be used to set the other parameters.

### Changing Reference Files <a class="anchor" id="3g"></a>

As shown above in example 1 (link), the `model.meta` (which is derived from the `fits` file header) contains paths to the reference files that will be used by the pipeline. Some users may want to substitute reference files - for example, to run with the old version of a particular file (not the entire set of files), or to use a custom made reference file. There are multiple options to do this when running the pipeline in Python.

In these examples, we will run `Detector1Pipeline` on the `uncal.fits`, but substitute the CRDS-chosen dark reference file for our own, `my_dark.fits`.

#### Option 1: Overriding Reference Files as an Argument to A Pipeline/Step<a class="anchor" id="3g1"></a>

The simplest way to substitute a reference file is to provide it as an argument when creating the pipeline/step instance. By setting it this way, this information will also be passed on when running the pipeline with `call`.


In [15]:
pipe1_mydarkref = Detector1Pipeline()
#pipe1_mydarkref('jw42424001001_01101_00001_nrca5_uncal.fits', dark_filename='my_dark.fits')

2021-05-10 09:50:52,081 - stpipe.Detector1Pipeline - INFO - Detector1Pipeline instance created.
2021-05-10 09:50:52,083 - stpipe.Detector1Pipeline.group_scale - INFO - GroupScaleStep instance created.
2021-05-10 09:50:52,085 - stpipe.Detector1Pipeline.dq_init - INFO - DQInitStep instance created.
2021-05-10 09:50:52,087 - stpipe.Detector1Pipeline.saturation - INFO - SaturationStep instance created.
2021-05-10 09:50:52,089 - stpipe.Detector1Pipeline.ipc - INFO - IPCStep instance created.
2021-05-10 09:50:52,091 - stpipe.Detector1Pipeline.superbias - INFO - SuperBiasStep instance created.
2021-05-10 09:50:52,093 - stpipe.Detector1Pipeline.refpix - INFO - RefPixStep instance created.
2021-05-10 09:50:52,094 - stpipe.Detector1Pipeline.rscd - INFO - RscdStep instance created.
2021-05-10 09:50:52,096 - stpipe.Detector1Pipeline.firstframe - INFO - FirstFrameStep instance created.
2021-05-10 09:50:52,098 - stpipe.Detector1Pipeline.lastframe - INFO - LastFrameStep instance created.
2021-05-10 0

#### Option 2: Overriding Reference Files on a Pipeline/Step Directly<a class="anchor" id="3g2"></a>
When calling the pipeline object directly using `pipe.run`, the desired reference file overrides can be set directly via the `override_<reffile>` attributes.



In [16]:
pipe1_mydarkref = Detector1Pipeline()
pipe1_mydarkref.dark_current.override_dark = 'my_dark.fits'
# result_mydarkref = pipe1_mydarkref('jw42424001001_01101_00001_nrca5_uncal.fits')

2021-05-10 09:50:56,282 - stpipe.Detector1Pipeline - INFO - Detector1Pipeline instance created.
2021-05-10 09:50:56,284 - stpipe.Detector1Pipeline.group_scale - INFO - GroupScaleStep instance created.
2021-05-10 09:50:56,286 - stpipe.Detector1Pipeline.dq_init - INFO - DQInitStep instance created.
2021-05-10 09:50:56,288 - stpipe.Detector1Pipeline.saturation - INFO - SaturationStep instance created.
2021-05-10 09:50:56,290 - stpipe.Detector1Pipeline.ipc - INFO - IPCStep instance created.
2021-05-10 09:50:56,292 - stpipe.Detector1Pipeline.superbias - INFO - SuperBiasStep instance created.
2021-05-10 09:50:56,294 - stpipe.Detector1Pipeline.refpix - INFO - RefPixStep instance created.
2021-05-10 09:50:56,295 - stpipe.Detector1Pipeline.rscd - INFO - RscdStep instance created.
2021-05-10 09:50:56,297 - stpipe.Detector1Pipeline.firstframe - INFO - FirstFrameStep instance created.
2021-05-10 09:50:56,299 - stpipe.Detector1Pipeline.lastframe - INFO - LastFrameStep instance created.
2021-05-10 0

### Skipping Steps <a class="anchor" id="3h"></a>

Sometimes you may want to run a pipeline, but omit certain steps contained within that pipeline. 

#### Option 1: Specifying Pipeline Steps to Skip within a Parameter File<a class="anchor" id="3h1"></a>

To skip the `LinearityStep`, add the following to the `Detector1Pipeline` parameter file: 

steps:

\- class: jwst.linearity.linearity_step.LinearityStep

  parameters:
  
       skip: true

#### Option 2: Skipping Steps Directly when using 'run'<a class="anchor" id="3h2"></a>

Just like setting parameteres or reference files, when calling a pipeline instance directly using the `run` method, directions to skip steps can be set as attributes. For example, to run `Detector1Pipeline` but skip the 

In [17]:
pipe_skipdark = Detector1Pipeline()
pipe_skipdark.dark_current.skip = True

2021-05-10 09:51:12,864 - stpipe.Detector1Pipeline - INFO - Detector1Pipeline instance created.
2021-05-10 09:51:12,866 - stpipe.Detector1Pipeline.group_scale - INFO - GroupScaleStep instance created.
2021-05-10 09:51:12,868 - stpipe.Detector1Pipeline.dq_init - INFO - DQInitStep instance created.
2021-05-10 09:51:12,870 - stpipe.Detector1Pipeline.saturation - INFO - SaturationStep instance created.
2021-05-10 09:51:12,871 - stpipe.Detector1Pipeline.ipc - INFO - IPCStep instance created.
2021-05-10 09:51:12,873 - stpipe.Detector1Pipeline.superbias - INFO - SuperBiasStep instance created.
2021-05-10 09:51:12,875 - stpipe.Detector1Pipeline.refpix - INFO - RefPixStep instance created.
2021-05-10 09:51:12,877 - stpipe.Detector1Pipeline.rscd - INFO - RscdStep instance created.
2021-05-10 09:51:12,879 - stpipe.Detector1Pipeline.firstframe - INFO - FirstFrameStep instance created.
2021-05-10 09:51:12,880 - stpipe.Detector1Pipeline.lastframe - INFO - LastFrameStep instance created.
2021-05-10 0

# Running the Pipeline with the Command Line Interface (CLI) <a class="anchor" id="4"></a>

The command line interface to the JWST pipeline is called `strun`. Everthing that is possible when running the pipeline in Python is also available via this interface - which one you use is up to personal preference. 

The basic formula to run the pipeline with the CLI is:

 `$ strun <pipeline_name, class_name, or parameter_file> <input_file>`


The full class name (i.e `jwst.pipeline.Detector1Pipeline`) of a pipeline/step, or a parameter file (from which the pipeline/step is implied) is the first argument, followed by the path to the input data. Pipeline classes also have an “alias”, or short name, that can be used instead of the full class specification. For example, `jwst.pipeline.Detector1Pipeline` has the alias `calwebb_detector1` and can be run as
### Example 3: Running a Full Pipeline using the CLI <a class="anchor" id="4a"></a>

The following is the command line equivilent to [Example 1](#3b). This will run the `Detector1Pipeline` on the uncal.fits file with the CRDS designated parameter file, and all best reference files chosen by CRDS.

In your shell:

    >> $ strun jwst.pipeline.Detector1Pipeline jw42424001001_01101_00001_nrca5_uncal.fits
   
    Or, so you don't have to type out the full class name, the alias name for this pipeline can be used.
    
    >> $ strun calwebb_detector1 jw42424001001_01101_00001_nrca5_uncal.fits

### Example 4: Running a Single Pipeline Step using the CLI <a class="anchor" id="4b"></a>

The CLI can also be used to run a single pipeline step. The equivilant to [Example 2](#3c) would be:

    >> $ strun jwst.linearity.LinearityStep jw42424001001_01101_00001_nrca5_uncal.fits

    
### Configuring the Pipeline using the Command Line Interface <a class="anchor" id="4c"></a>
### Setting Parameters <a class="anchor" id="4d"></a>

Parameters can only be changed via parameter file when running with the command line interface. To use a custom parameter file, instead of the class name, use the path to the parameter file. For example:

    >> strun my_custom_parfile_tweakregstep.asdf jw42424001001_01101_00001_nrca5_uncal.fits
#### Changing Reference Files <a class="anchor" id="4e"></a>

To override a reference file, simply add a flag to the `strun` call. For example, to override the CRDS-chosed dark file with your own in the same working directory as the file.

    >> $ strun calwebb_detector1 jw42424001001_01101_00001_nrca5_uncal.fits
      --steps.dark_current.override_dark='my_dark.fits'

####  Skipping Steps <a class="anchor" id="4f"></a>

Pipeline steps can also be skipped by adding a flag to the call. For example, to skip `LinearityStep` in the `Detector1Pipeline` - this is equivalent to the examples shown in the [Python section](#3h)
    >> $ strun calwebb_detector1 jw42424001001_01101_00001_nrca5_uncal.fits
    --steps.linearity.skip=True
    
#### Additional Command Line Functionality <a class="anchor" id="4g"></a>

In addition to 'strun', the command line interface has tools to inspect avaiable pipelines and their parameters.

To see what parameters are available for any given pipeline or step, use the -h option on strun. For example:

    >> $ strun calwebb_detector1 -h
    >> $ strun jwst.dq_init.DQInitStep -h
    
To see all available pipelines, steps, and their aliases:

    >> $ stpipe list
    